In [3]:
# keras imports for the dataset and building our neural network
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

# from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from matplotlib.gridspec import GridSpec
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from tensorflow.keras import callbacks
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPool2D,
)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay




In [3]:
def display_classification_report(
    classification_report, figure_path, figure_name, onscreen=True
):
    f = open(os.path.join(figure_path, figure_name + ".txt"), "w")
    f.write(classification_report)
    f.close()

    if onscreen:
        print(classification_report)

In [4]:
def display_confusion_matrix(
    confusion_matrix, labels, figure_path, figure_name, figure_format, onscreen=True
):
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

    disp.plot(cmap=plt.cm.Greys)
    fig = disp.figure()

    plt.savefig(
        os.path.join(figure_path, figure_name + "." + figure_format),
        format=figure_format,
    )

    if onscreen:
        print("Show confusion matrix on display")
        plt.show()
    else:
        plt.close(fig)


def display_activations(
    input,
    label,
    activations,
    layer_names,
    figure_path,
    figure_name,
    figure_format,
    onscreen=True,
):
    fig = plt.figure(layout="constrained", figsize=(10, 8))
    subfigs = fig.subfigures(
        1, len(activations) + 1
    )  # layers + input , figsize=(row_size*2.5,len(model.layers)*1.5))

    subfigs[0].subplots(1, 1)
    subfigs[0].suptitle("Label: {}".format(label))
    axs = subfigs[0].get_axes()
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[0].imshow(input, cmap="gray_r")

    for layer_index in range(0, len(activations)):
        print("layer:" + str(layer_index))
        print(activations[layer_index].shape[-1])
        subfigs[layer_index + 1].suptitle(layer_names[layer_index])
        subfigs[layer_index + 1].subplots(activations[layer_index].shape[-1], 1)

    for layer_index in range(0, len(activations)):
        print(activations[layer_index].shape)
        # range(0,activations.shape[-1]):
        axs = subfigs[layer_index + 1].get_axes()
        for plane_index in range(0, activations[layer_index].shape[-1]):
            plane = activations[layer_index][0, :, :, plane_index]
            # activations -= activations.mean()
            # activations /= activations.std()
            # activations *= 64
            # activations += 128
            # activations = np.clip(activations, 0, 255).astype('uint8')
            # axs[plane_index].imshow(model.layers[layer_index].output[0, :, :, plane_index], cmap='gray')
            axs[plane_index].set_xticks([])
            axs[plane_index].set_yticks([])
            axs[plane_index].imshow(plane, cmap="gray_r")

    fig.savefig(
        os.path.join(figure_path, figure_name + "." + figure_format),
        format=figure_format,
    )
    if onscreen:
        print("Show activations on display")
        plt.show()
    else:
        plt.close(fig)


def display_weights_column(
    weights, layer_names, figure_path, figure_name, figure_format, onscreen=True
):
    n_layers_with_weights = 0
    for layer_index in range(0, len(weights)):
        layer_weights = weights[layer_index]
        if len(layer_weights) > 0:
            n_layers_with_weights += 1

    fig = plt.figure(layout="constrained", figsize=(30, 15), frameon=False)
    # fig.tight_layout()
    plt.subplots_adjust(top=0.1, bottom=0.1, hspace=0.1, wspace=0.1)
    subfigs = fig.subfigures(1, n_layers_with_weights)
    layer_index_with_weights = 0
    print("Number of layers: " + str(len(weights)))
    for layer_index in range(0, len(weights)):
        layer_weights = weights[layer_index]

        print("layer:" + str(layer_index))
        # only weights (0) no biases (1)
        # print(len(layer_weights[0]))
        # print(weights[layer_index])
        # for i in range(0,len(weights[layer_index])):
        # subfigs[layer_index+1].suptitle(layer_names[layer_index])
        # only weights no biases
        if len(layer_weights) > 0 and len(layer_weights[0].shape) > 1:
            print(" weights shape ", layer_weights[0].shape)

            # subfigs[layer_index_with_weights].suptitle(layer_names[layer_index])
            # subfigs[layer_index_with_weights].tight_layout()
            # squeeze=False squeezing at all is done: the returned Axes object is always a 2D array containing
            axs = subfigs[layer_index_with_weights].subplots(
                layer_weights[0].shape[-2], layer_weights[0].shape[-1], squeeze=False
            )  # , sharex=True, sharey=True)
            subfigs[layer_index_with_weights].subplots_adjust(
                wspace=0.001, hspace=0.0, top=0.0, bottom=0.0, left=0.0, right=0.0
            )
            print(axs.shape)
            for i in range(0, layer_weights[0].shape[-2]):
                for j in range(0, layer_weights[0].shape[-1]):
                    # print(i,j)
                    w = layer_weights[0]
                    # print(w[:,:,i,j])
                    axs[i, j].imshow(
                        w[:, :, i, j], cmap="gray_r", interpolation="nearest"
                    )
                    axs[i, j].axis("off")
                    # axs[i,j].set_xticks([])
                    # axs[i,j].set_yticks([])
                    # axs[i,j].imshow(w[:,:,i,j], cmap='gray_r', interpolation='nearest')

            layer_index_with_weights += 1
    fig.savefig(
        os.path.join(figure_path, figure_name + "." + figure_format),
        format=figure_format,
    )

    if onscreen:
        print("Show weights on display")
        plt.show()
    else:
        plt.close(fig)


def display_loss_function(
    history, figure_path, figure_name, figure_format, onscreen=True
):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)
    fig = plt.figure()
    plt.plot(epochs, loss, color="red", label="Training loss")
    plt.plot(epochs, val_loss, color="green", label="Validation loss")
    plt.title("Training loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    fig.savefig(
        os.path.join(figure_path, figure_name + "." + figure_format),
        format=figure_format,
    )
    if onscreen:
        print("Show loss on display")
        plt.show()
    else:
        plt.close(fig)

In [5]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# let's print the shape of the dataset

# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)


# normalizing the data
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = to_categorical(y_train, n_classes)
Y_test = to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_train shape (60000, 28, 28, 1)
y_train shape (60000,)
X_test shape (10000, 28, 28, 1)
y_test shape (10000,)
Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


In [6]:
n_cnn1planes = 15
n_cnn1kernel = 3
n_poolsize = 1


# Stride defines the step size at which the filter moves across the input during convolution.
# A larger stride results in a reduction of the spatial dimensions of the output feature map.
# Stride can be adjusted to control the level of downsampling in the network.
# Stride is a critical parameter for controlling the spatial resolution of the feature maps and influencing the receptive field of the network.
n_strides = 1
n_dense = 100
dropout = 0.3

n_epochs = 10

model_name = (
    "CNN_Handwritten_OCR_CNN"
    + str(n_cnn1planes)
    + "_KERNEL"
    + str(n_cnn1kernel)
    + "_Epochs"
    + str(n_epochs)
)
# figure_format='svg'
figure_format = "png"
figure_path = "./"
log_path = "./log"

# layer_outputs = [layer.output for layer in model.layers[1:7]]
# activation_model = Model(inputs=model.input,outputs=layer_outputs)

# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
cnn1 = Conv2D(
    n_cnn1planes,
    kernel_size=(n_cnn1kernel, n_cnn1kernel),
    strides=(n_strides, n_strides),
    padding="valid",
    activation="relu",
    input_shape=(28, 28, 1),
)
model.add(cnn1)
model.add(MaxPool2D(pool_size=(n_poolsize, n_poolsize)))

# model.add(Dropout(dropout))

cnn2 = Conv2D(
    n_cnn1planes * 2,
    kernel_size=(n_cnn1kernel, n_cnn1kernel),
    strides=(n_strides, n_strides),
    padding="valid",
    activation="relu",
)
model.add(cnn2)
model.add(MaxPool2D(pool_size=(n_poolsize, n_poolsize)))

# model.add(Dropout(dropout))

cnn3 = Conv2D(
    n_cnn1planes * 4,
    kernel_size=(n_cnn1kernel, n_cnn1kernel),
    strides=(n_strides, n_strides),
    padding="valid",
    activation="relu",
)
model.add(cnn3)
model.add(MaxPool2D(pool_size=(n_poolsize, n_poolsize)))

# model.add(Dropout(dropout))

# flatten output of conv
model.add(Flatten())


model.add(Dropout(dropout))

# hidden layer
model.add(Dense(n_dense, activation="relu"))
# output layer
model.add(Dense(n_classes, activation="softmax"))

# compiling the sequential model

model_name += "_Optimzer_" + "SGD"

# vary the constant learning rate
model_name += "_LearningRate_" + "Constant"
learning_rate = 0.001

# OR use a learning rate scheduler that adapts the learning rate over the epochs of the training process
# https://keras.io/2.15/api/optimizers/learning_rate_schedules/

# model_name += '_LearningRate_' + 'ExponentialDecay'
# learning_rate = ExponentialDecay(initial_learning_rate=1e-2, decay_steps=n_epochs, decay_rate=0.9)

# learning_rate=0.01
momentum = 0.9
optimizer = SGD(learning_rate=learning_rate, momentum=momentum)

# optimizer=Adam(learning_rate = learning_rate)


# vary the constant learning rate
# learning_rate = 0.01
# optimizer=SGD(learning_rate=learning_rate)

model.compile(
    loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer
)


layer_names = [layer.name for layer in model.layers[:8]]

weights = [layer.get_weights() for layer in model.layers[:4]]
figure_name = model_name + "_initial_weights"
display_weights_column(weights, layer_names, "./", figure_name, figure_format, False)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
<ipython-input-4-2d6111f7471d>:87: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  plt.subplots_adjust(top=0.1, bottom=0.1, hspace=0.1, wspace=0.1)


Number of layers: 4
layer:0
 weights shape  (3, 3, 1, 15)


<ipython-input-4-2d6111f7471d>:110: UserWarning: This figure was using a layout engine that is incompatible with subplots_adjust and/or tight_layout; not calling subplots_adjust.
  subfigs[layer_index_with_weights].subplots_adjust(


(1, 15)
layer:1
layer:2
 weights shape  (3, 3, 15, 30)
(15, 30)
layer:3


In [7]:
# training the model for n_epochs, use 10% of the training data as validation data
history = model.fit(
    X_train,
    Y_train,
    validation_split=0.1,
    batch_size=128,
    epochs=n_epochs,
    #callbacks=[callbacks.TensorBoard()],
)

Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 225s 529ms/step - accuracy: 0.6206 - loss: 1.1815 - val_accuracy: 0.9253 - val_loss: 0.2382
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 256s 515ms/step - accuracy: 0.9179 - loss: 0.2717 - val_accuracy: 0.9598 - val_loss: 0.1474
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 257s 505ms/step - accuracy: 0.9401 - loss: 0.2013 - val_accuracy: 0.9663 - val_loss: 0.1225
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 268s 518ms/step - accuracy: 0.9527 - loss: 0.1574 - val_accuracy: 0.9708 - val_loss: 0.1042
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 260s 513ms/step - accuracy: 0.9596 - loss: 0.1378 - val_accuracy: 0.9750 - val_loss: 0.0907
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 267s 526ms/step - accuracy: 0.9676 - loss: 0.1070 - val_accuracy: 0.9778 - val_loss: 0.0857
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 219s 519ms/step - accuracy: 0.9702 - loss: 0.0980 - val_accuracy: 0.9793 - val_loss: 0.0802
Epoch 8/10
 70/422 ━━━━━━━━━━━━━━━━━━━━ 2:51 488ms/step - accuracy: 0.9734 -

KeyboardInterrupt: 

In [ ]:
figure_name = model_name + "_loss"
display_loss_function(history, "./", figure_name, figure_format)


weights = [layer.get_weights() for layer in model.layers[:4]]
figure_name = model_name + "_weights"
display_weights_column(weights, layer_names, "./", figure_name, figure_format, False)

X_test_images = X_test[:2]
for i in range(X_test_images.shape[0]):
    Y_test_pred = model.predict(np.expand_dims(X_test[i], axis=0))
    # ax.set_title('Label: {}'.format(np.argmax(Y_test[i])))

    activation_model = Model(
        inputs=model.inputs, outputs=[layer.output for layer in model.layers]
    )
    activations = activation_model.predict(np.expand_dims(X_test[i], axis=0))

    figure_name = model_name + "_activations_" + "test_image_" + str(i)
    display_activations(
        X_test[i],
        np.argmax(Y_test[i]),
        activations[:4],
        layer_names[:4],
        figure_path,
        figure_name,
        figure_format,
    )

y_test_pred = model.predict(X_test)
cm = confusion_matrix(
    y_test, np.argmax(y_test_pred, axis=1), labels=range(0, n_classes)
)

figure_name = model_name + "_confusion_matrix"
display_confusion_matrix(
    cm, range(0, n_classes), figure_path, figure_name, figure_format
)

figure_name = model_name + "_classification_report"
display_classification_report(
    classification_report(
        y_test,
        np.argmax(y_test_pred, axis=1),
        target_names=[str(c) for c in range(0, n_classes)],
        digits=4,
    ),
    figure_path,
    figure_name,
)

figure_name = model_name + "_model_summary"
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
model_summary = "\n".join(stringlist)
display_classification_report(model_summary, figure_path, figure_name)